In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=5min&apikey=XX5ESBW86BXRZI0E'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data['Time Series (5min)']).transpose()

In [ ]:

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&month=2023-11&outputsize=full&symbol=MSFT&interval=5min&apikey=XX5ESBW86BXRZI0E'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
df11 = pd.DataFrame(data['Time Series (5min)']).transpose()

In [ ]:
df11

In [ ]:
df.sort_index(inplace=True, ascending=True)

In [ ]:
df11[:200]

In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=XX5ESBW86BXRZI0E'
r = requests.get(url)
data = r.json()

print(data)

In [ ]:
df = pd.DataFrame(data['Time Series (Daily)']).transpose()

In [ ]:
df

In [ ]:
sns.lineplot(data=df,x=df.index,
             y='4. close')

In [ ]:
list(df['4. close'])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
sns.lineplot(data=df11[200:2000],x=df11.index[200:2000],
             y='4. close')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df11[:100]

In [ ]:
df11.index 

In [ ]:
import akshare as ak

stock_us_daily_df = ak.stock_us_daily(symbol="MSFT", adjust="qfq")
print(stock_us_daily_df)

In [ ]:
stock_us_daily_df[-100:]

In [ ]:
import akshare as ak

stock_us_famous_spot_em_df = ak.stock_us_famous_spot_em(symbol='科技类')
print(stock_us_famous_spot_em_df)

In [ ]:
import akshare as ak

us_stock_current_df = ak.stock_us_spot()
print(us_stock_current_df)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../2023-11-21-America-sotck-list.csv', header=0, index_col=0)

In [ ]:
df.sche

In [ ]:
df.ipoDate

In [ ]:
df.columns

In [ ]:
df.delistingDatedf

In [ ]:
df.ipoDate = pd.to_datetime(df.ipoDate)
df.delistingDate = pd.to_datetime(df.delistingDate)

In [ ]:
print(pd.io.sql.get_schema(df, name='stock_meta_data'))

In [ ]:
df['country'] = 'US'

In [ ]:
create_db = pd.io.sql.get_schema(df, name='stock_meta_data')
print()

In [ ]:
insp = db.inspect(db_engine)

In [ ]:
table_name="american_stock_list"

In [ ]:
insp.has_table(table_name)

In [ ]:
db_engine.execute(db.text(create_db))

In [ ]:
with db_engine.connect() as c:
    result = c.execute(db.text(create_db))
    for row in result:
        print(rwo)

In [ ]:
db.__version__

In [ ]:
from prefect.filesystems import RemoteFileSystem
import pandas as pd
import io
import sqlalchemy as db
import datetime

In [ ]:
fs = await RemoteFileSystem.load("stock-meta")

In [ ]:
x = await fs.read_path('2023-11-26-America-sotck-list.csv')

In [ ]:
df['country'] = 'US'
df['last_update'] = datetime.datetime.now()

In [ ]:
df['delistingDate'].where(df['delistingDate'].notnull(), None)

In [ ]:
df['delistingDate'].where(df['delistingDate'].notnull(), None)

In [ ]:
df['delistingDate'].notnull()

In [ ]:
df.head()

In [ ]:
df = pd.read_csv(io.BytesIO(x), index_col=0, header=0)

In [ ]:
df.ipoDate = pd.to_datetime(df.ipoDate)
df.delistingDate = pd.to_datetime(df.delistingDate)
print(pd.io.sql.get_schema(df, name='stock_meta'))

In [ ]:
# create postgres db engine
user = "root"
password = "root"
host = "localhost"
database = "q_data"
connect_str = f"postgresql://{user}:{password}@{host}:5432/{database}"
print(connect_str)
# db_engine = db.create_engine()


In [ ]:
with db_engine.connect() as conn:
    conn.execute(db.text("""
        CREATE TABLE IF NOT EXISTS stock_meta (
            symbol TEXT PRIMARY KEY,
            name TEXT,
            exchange TEXT,
            assetType TEXT,
            ipoDate TIMESTAMP,
            delistingDate TIMESTAMP,
            status TEXT,
            country TEXT,
            last_update TIMESTAMP)
    """))
    conn.commit()

with db_engine.connect() as conn: 
    for index, item in df.fillna('NULL').iterrows(): 
        delistingDate = f"'{item.delistingDate}'" if item.delistingDate != 'NULL' else 'NULL'
        insert_stmt = db.text(f"""
            INSERT INTO stock_meta (
                symbol,
                name, 
                exchange, 
                assetType,
                ipoDate,
                delistingDate,
                status,
                country,
                last_update
            ) VALUES (
                '{item.symbol}',
                '{item['name'].replace("'", "''")}',
                '{item.exchange}',
                '{item.assetType}',
                '{item.ipoDate}',
                {delistingDate},
                '{item.status}',
                '{item.country}',
                '{item.last_update}'
            )
            ON CONFLICT (symbol) DO UPDATE
            SET
                delistingdate = EXCLUDED.delistingDate,
                status = EXCLUDED.status,
                last_update = EXCLUDED.last_update
            WHERE
                stock_meta.delistingdate <> EXCLUDED.delistingDate OR
                stock_meta.delistingdate IS NULL;
        """)
        conn.execute(insert_stmt)
    
    conn.commit()



    

In [ ]:
def insert_one_row(connect, item):
    delistingDate = f"'{item.delistingDate}'" if item.delistingDate != 'NULL' else 'NULL'
    insert_stmt = db.text(f"""
        INSERT INTO stock_meta (
            symbol,
            name, 
            exchange, 
            assetType,
            ipoDate,
            delistingDate,
            status,
            country,
            last_update
        ) VALUES (
            '{item.symbol}',
            '{item['name'].replace("'", "''")}',
            '{item.exchange}',
            '{item.assetType}',
            '{item.ipoDate}',
            {delistingDate},
            '{item.status}',
            '{item.country}',
            '{item.last_update}'
        )
        ON CONFLICT (symbol) DO UPDATE
        SET
            delistingdate = EXCLUDED.delistingDate,
            status = EXCLUDED.status,
            last_update = EXCLUDED.last_update
        WHERE
            stock_meta.delistingdate <> EXCLUDED.delistingDate OR
            stock_meta.delistingdate IS NULL;
    """)
    print(str(insert_stmt))
    connect.execute(insert_stmt)

In [ ]:
x = df.iloc[0]
x.delistingDate = datetime.datetime.now()

In [ ]:
_, t = next(df.iterrows())

In [ ]:
x.last_update = datetime.datetime.now()

In [ ]:
with db_engine.connect() as conn: 
    insert_one_row(conn, x)
    conn.commit()

In [ ]:
t['name']

In [ ]:
df.delistingDate.where(df.delistingDate.notnull(), 'NULL')